# 04 - Obchodní doporučení

## Cíl tohoto notebooku

V závěrečném kroku workshopu navrhneme sázkové kurzy pro naši novou sázkovou kancelář.

**Co se naučíte:**
- Výpočet sázkových kurzů z pravděpodobnosti
- Kategorizaci týmů pomocí kvantilů
- Implementaci vlastních funkcí pro analýzu
- Kritické hodnocení vlastní analýzy

**Vstup:** Soubor `data/processed/hockey_teams.csv`

---

## Teorie: Jak fungují sázkové kurzy

Sázkový kurz je převrácenou hodnotou pravděpodobnosti. Pokud je pravděpodobnost výhry týmu 50% (0.5), kurz bude:

**kurz = 1 / pravděpodobnost = 1 / 0.5 = 2.0**

To znamená, že pokud vsadíte 100 Kč na kurz 2.0, při výhře dostanete 200 Kč.

| Pravděpodobnost | Kurz |
|-----------------|------|
| 100% (1.0) | 1.0 |
| 50% (0.5) | 2.0 |
| 25% (0.25) | 4.0 |
| 10% (0.1) | 10.0 |

Více informací: [Kursy bukmacherskie](https://trustbet.pl/kursy-bukmacherskie/)

---

## Krok 1: Příprava prostředí

### 1.1 Import knihoven

In [3]:
# Importujte pandas, případně další potřebné knihovny
import pandas as pd

### 1.2 Načtení dat

In [4]:
# Načtěte CSV soubor z data/processed/hockey_teams.csv
# Nezapomeňte na správný oddělovač (;)
df = pd.read_csv('../data/processed/hockey_teams.csv', sep = ';')


<details>
<summary>Nápověda</summary>
`pd.read_read_csv()`
</details>

### 1.3 Kontrola dat

In [5]:
# Zkontrolujte, že data byla správně načtena - napr. počet rádků
df.shape


(582, 10)

---

## Krok 2: Implementace funkce pro výpočet kurzu

### 2.1 Definice funkce get_betting_odds

In [6]:
df.head(1)

,team,season,victories,defeats,overtime_defeats,victory_percentage,scored_goals,received_goals,goal_difference,goals_ratio
0,Boston Bruins,1990,44,24,0,0.55,299,264,35,1.132576


In [10]:
def get_betting_odds(probability):
    """
    Vypočítá sázkový kurz z pravděpodobnosti.
    
    Parametry:
        probability: Pravděpodobnost události (0-1)
    
    Vrací:
        Decimální sázkový kurz
    
    Příklad:
        get_betting_odds(0.5) -> 2.0
    """
    return 1 / probability

<details>
<summary>Nápověda</summary>
1 / probability    
</details>

### 2.2 Testy funkce

In [11]:
# Spusťte testy pro ověření správnosti implementace

def test_get_betting_odds():
    assert get_betting_odds(1) == 1, "Očekáváno 1"
    assert get_betting_odds(0.5) == 2, "Očekáváno 2"
    assert get_betting_odds(0.25) == 4, "Očekáváno 4"
    assert get_betting_odds(0.1) == 10, "Očekáváno 10"
    try:
        get_betting_odds(0)
    except ZeroDivisionError:
        pass
    else:
        assert False, "Očekávána ZeroDivisionError"
    
    print("Všechny testy prošly!")

test_get_betting_odds()

Všechny testy prošly!


---

## Krok 3: Výpočet globálního kurzu

### 3.1 Určení globální pravděpodobnosti výhry

Vypočítejte průměrnou pravděpodobnost výhry libovolného týmu na základě historických dat.

In [17]:
# Vypočítejte průměrnou hodnotu victory_percentage
# To je naše globální pravděpodobnost výhry
prob = df['victory_percentage'].mean()

print(f'Pravdepodobnost vyhry libovolneho tymu: {100*prob:.2f}%')


Pravdepodobnost vyhry libovolneho tymu: 45.85%


<details>
<summary>Nápověda</summary>

```python
globalni_pravdepodobnost = df['victory_percentage'].mean()
print(f"Globální pravděpodobnost výhry: {globalni_pravdepodobnost:.4f}")
print(f"To odpovídá: {globalni_pravdepodobnost * 100:.2f}%")
```
</details>

### 3.2 Výpočet globálního kurzu

In [19]:
# Použijte funkci get_betting_odds pro výpočet kurzu
# Zaokrouhlete na 2 desetinná místa

print(f'Stavkovy kurz je: {get_betting_odds(prob):.2f}')

Stavkovy kurz je: 2.18


<details>
<summary>Nápověda</summary>

```python
globalni_kurz = round(get_betting_odds(globalni_pravdepodobnost), 2)
print(f"Globální sázkový kurz: {globalni_kurz}")
```
</details>

---

## Krok 4: Kategorizace týmů do lig

### 4.1 Definice kategorií

Rozdělíme týmy do 4 kategorií podle jejich výkonnosti:

| Liga | Popis | Kritérium |
|------|-------|----------|
| A | Nejlepší týmy | Top 5% |
| B | Dobré týmy | 70-95 percentil |
| C | Průměrné týmy | 20-70 percentil |
| D | Nejslabší týmy | Spodních 20% |

### 4.2 Výpočet hranic percentilů

In [22]:
df['victory_percentage'].describe(percentiles =[0, 0.20, 0.70, 0.95])

count    582.000000
mean       0.458519
std        0.102237
min        0.119000
0%         0.119000
20%        0.366000
50%        0.463000
70%        0.512000
95%        0.621850
max        0.756000
Name: victory_percentage, dtype: float64

In [25]:
# Vypočítejte hranice pro jednotlivé kategorie
# Použijte quantile() na sloupci victory_percentage
limit_a = df['victory_percentage'].quantile(0.95)
limit_b = df['victory_percentage'].quantile(0.70)
limit_c = df['victory_percentage'].quantile(0.20)

print(f"Hranice pro A ligu je: {limit_a:.4f}")
print(f"Hranice pro B ligu je: {limit_b:.4f}")
print(f"Hranice pro C ligu je: {limit_c:.4f}")

Hranice pro A ligu je: 0.6218
Hranice pro B ligu je: 0.5120
Hranice pro C ligu je: 0.3660


<details>
<summary>Nápověda</summary>

```python
hranice_d = df['victory_percentage'].quantile(0.20)  # Spodních 20%
hranice_c = df['victory_percentage'].quantile(0.70)  # 20-70%
hranice_b = df['victory_percentage'].quantile(0.95)  # 70-95%
# Vše nad 95% je liga A

print(f"Liga D: victory_percentage < {hranice_d:.4f}")
print(f"Liga C: {hranice_d:.4f} <= victory_percentage < {hranice_c:.4f}")
print(f"Liga B: {hranice_c:.4f} <= victory_percentage < {hranice_b:.4f}")
print(f"Liga A: victory_percentage >= {hranice_b:.4f}")
```
</details>

### 4.3 Implementace funkce pro přiřazení ligy

In [26]:
def assign_team_to_league(victory_pct, limit_a, limit_b, limit_c):
    """
    Přiřadí tým do ligy podle procenta výher.
    
    Parametry:
        victory_pct: Procento výher týmu
        limit_a, limit_b, limit_c: limity pro ligy
    
    Vrací:
        Písmeno ligy (A, B, C nebo D)
    """
        # Použijte hranice vypočítané výše
    if victory_pct >= limit_a:
        return "A"
    elif victory_pct >= limit_b:
        return "B"
    elif victory_pct >= limit_c:
        return "C"
    else:
        return "D"

<details>
<summary>Nápověda</summary>

```python
def assign_team_to_league(victory_pct):
    if victory_pct >= hranice_b:
        return 'A'
    elif victory_pct >= hranice_c:
        return 'B'
    elif victory_pct >= hranice_d:
        return 'C'
    else:
        return 'D'
```
</details>

### 4.4 Aplikace na dataset

In [29]:
# Vytvořte nový sloupec 'league' pomocí apply()
df['league'] = df['victory_percentage'].apply(lambda x : assign_team_to_league(x, limit_a, limit_b, limit_c))


In [31]:
df.groupby('league')['league'].count()

league
A     30
B    172
C    277
D    103
Name: league, dtype: int64

<details>
<summary>Nápověda</summary>

```python
df['league'] = df['victory_percentage'].apply(assign_team_to_league)

# Kontrola distribuce
print("Počet týmů v jednotlivých ligách:")
print(df['league'].value_counts().sort_index())
```
</details>

In [32]:
print(df['league'].value_counts().sort_index())

league
A     30
B    172
C    277
D    103
Name: count, dtype: int64


---

## Krok 5: Výpočet kurzů pro jednotlivé ligy

### 5.1 Průměrná pravděpodobnost podle ligy

In [42]:
# Spočítejte průměrné victory_percentage pro každou ligu
mean_league = df.groupby('league')['victory_percentage'].mean()
mean_league

league
A    0.642500
B    0.551384
C    0.439686
D    0.300505
Name: victory_percentage, dtype: float64

<details>
<summary>Nápověda</summary>

```python
prumer_liga = df.groupby('league')['victory_percentage'].mean()
print("Průměrná pravděpodobnost výhry podle ligy:")
print(prumer_liga.sort_index())
```
</details>

### 5.2 Kurzy pro jednotlivé ligy

In [43]:
# Vypočítejte kurz pro každou ligu
# Použijte funkci get_betting_odds()

bet_league = get_betting_odds(df.groupby('league')['victory_percentage'].mean())
bet_league

league
A    1.556420
B    1.813619
C    2.274351
D    3.327733
Name: victory_percentage, dtype: float64

<details>
<summary>Nápověda</summary>

```python
kurzy_liga = prumer_liga.apply(get_betting_odds).round(2)

print("\nSázkové kurzy podle ligy:")
for liga in ['A', 'B', 'C', 'D']:
    print(f"Liga {liga}: kurz {kurzy_liga[liga]}")
```
</details>

### 5.3 Kontrola rozumnosti výsledků

Zamyslete se:
- Jsou kurzy rozumné? (Liga A by měla mít nižší kurz, Liga D vyšší)
- Odpovídá to sportovní logice?

In [38]:
[df[df['league'] == l].shape[0] for l in ['A', 'B', 'C', 'D']]

[30, 172, 277, 103]

In [44]:
[mean_league[l] for l in ['A', 'B', 'C', 'D']]

[np.float64(0.6425),
 np.float64(0.5513837209302326),
 np.float64(0.43968592057761735),
 np.float64(0.30050485436893204)]

In [45]:
[bet_league[l] for l in ['A', 'B', 'C', 'D']]

[np.float64(1.556420233463035),
 np.float64(1.813619013475611),
 np.float64(2.2743507426535188),
 np.float64(3.327733264409408)]

In [48]:
pd.DataFrame({
    'League' : ['A', 'B', 'C', 'D'],
    'Teams' : [df[df['league'] == l].shape[0] for l in ['A', 'B', 'C', 'D']],
    'Victory probability' : [mean_league[l] for l in ['A', 'B', 'C', 'D']],
    'Bet' : [bet_league[l] for l in ['A', 'B', 'C', 'D']]
})

,League,Teams,Victory probability,Bet
0,A,30,0.642500,1.556420
1,B,172,0.551384,1.813619
2,C,277,0.439686,2.274351
3,D,103,0.300505,3.327733


<details>
<summary>Nápověda</summary>

```python
vysledky = pd.DataFrame({
    'Liga': ['A', 'B', 'C', 'D'],
    'Pocet_zaznamu': [df[df['league'] == l].shape[0] for l in ['A', 'B', 'C', 'D']],
    'Prumer_victory_pct': [prumer_liga[l] for l in ['A', 'B', 'C', 'D']],
    'Kurz': [kurzy_liga[l] for l in ['A', 'B', 'C', 'D']]
})

print("\nSouhrn výsledků:")
print(vysledky.to_string(index=False))
```
</details>

---

## Krok 6: Diskuze a kritické hodnocení

Před použitím výsledků v praxi je nutné kriticky zhodnotit analýzu.

### Otázky k diskuzi:

**1. Jaké prvky analýzy byly zjednodušeny? Co bylo vynecháno?**

*Zde napište své poznámky:*

- 
- 
- 

**2. Existují nějaké nesrovnalosti v odhadnutých kurzech?**

*Zde napište své poznámky:*

- 
- 
- 

**3. Jak lze zlepšit odhady kurzů?**

*Zde napište své poznámky:*

- 
- 
- 

**4. Jaká další data by pomohla zpřesnit analýzu?**

*Zde napište své poznámky:*

- 
- 
- 

**5. Jak lze simulovat výsledky analýzy pro ověření, že nevedou k finančním ztrátám?**

*Zde napište své poznámky:*

- 
- 
- 

---

## Shrnutí workshopu

V tomto workshopu jste prošli celým procesem datové analýzy:

1. **Stažení dat** - Použití Selenium pro web scraping
2. **Zpracování dat** - Parsování HTML pomocí BeautifulSoup, export do JSON
3. **Analýza dat** - Explorativní analýza pomocí pandas a matplotlib
4. **Obchodní doporučení** - Návrh sázkových kurzů

**Klíčové dovednosti:**
- Automatizace stahování dat z webu
- Transformace nestrukturovaných dat do strukturovaného formátu
- Čištění a příprava dat pro analýzu
- Vizualizace a interpretace výsledků
- Kritické hodnocení vlastní práce

---

## Tipy pro vylepšení

1. **Použití pandas.cut() pro kategorizaci:**
```python
# Elegantnější způsob kategorizace
df['league'] = pd.cut(
    df['victory_percentage'],
    bins=[0, hranice_d, hranice_c, hranice_b, 1],
    labels=['D', 'C', 'B', 'A']
)
```

2. **Přidání marže sázkové kanceláře:**
```python
def get_betting_odds_with_margin(probability, margin=0.05):
    """Kurz s marží sázkové kanceláře."""
    fair_odds = 1 / probability
    return fair_odds * (1 - margin)
```

3. **Validace modelu pomocí backtestingu:**
```python
# Rozdělení dat na tréninkovou a testovací sadu
train = df[df['season'] < 2010]
test = df[df['season'] >= 2010]

# Výpočet kurzů na tréninkových datech
# Validace na testovacích datech
```

4. **Ukládání konfigurace do slovníku:**
```python
CONFIG = {
    'percentily': {
        'A': 0.95,
        'B': 0.70,
        'C': 0.20
    },
    'marze': 0.05
}
```